Jon V

6/3/18

In [19]:
from ase.spacegroup import crystal
from ase.build import *
import numpy as np
import matplotlib.pyplot as plt
import os
import ase
import json

### Choose inputs.

In [20]:
# choose scf parameters
pref = 'benzene'
pseudo_dir = '/n/home03/jonpvandermause/qe-6.2.1/pseudo'
outdir='/n/home03/jonpvandermause/Cluster/Benzene_SCF'

# choose md parameters
dt = 20 # MD time step in Rydberg atomic units
nstep = 1000 # number of MD steps
pert_size = 0.01 * 5 # size of initial perturbation

# specify locations of QE codes
npool = 1 # number of k-pt pools
mult = 32
ncpu = npool * mult # number of cores
nodes = int(np.ceil(ncpu/32)) # number of nodes
pw_loc = '/n/home03/jonpvandermause/qe-6.2.1/bin/pw.x'
in_name = 'benzene.md.in'
out_name = 'benzene.md.out'
sh_name = 'Benzene_MD.sh'
partition = 'kozinsky'
memory = 1000
email = 'jonathan_vandermause@g.harvard.edu'

### Get perturbed positions.

In [21]:
import copy

# original positions of benzene atoms, taken from QE TDDFT benzene example
orig_pos = [['C',[5.633200899, 6.320861303, 5.]],
            ['C',[6.847051545, 8.422621957, 5.]],
            ['C',[8.060751351, 7.721904557, 5.]],
            ['C',[8.060707879, 6.320636665, 5.]],
            ['C',[6.846898786, 5.620067381, 5.]],
            ['C',[5.633279551, 7.722134449, 5.]],
            ['H',[6.847254360, 9.512254789, 5.]],
            ['H',[9.004364510, 8.266639340, 5.]],
            ['H',[9.004297495, 5.775895755, 5.]],
            ['H',[6.846845929, 4.530522778, 5.]],
            ['H',[4.689556006, 5.776237709, 5.]],
            ['H',[4.689791688, 8.267023318, 5.]]
           ]

# perturb the positions for MD run
struc_pert = copy.deepcopy(orig_pos)

for n in range(len(struc_pert)):
    for m in range(3):
        # get current coordinate
        coord_curr = struc_pert[n][1][m]

        # get perturbation by drawing from uniform
        pert = np.random.uniform(-pert_size, pert_size)

        # perturb the coordinate
        struc_pert[n][1][m] += pert


In [22]:
# get position text
def get_position_txt(struc):
    
    # write atomic positions
    postxt = ''
    postxt += 'ATOMIC_POSITIONS {angstrom}'
    for index, positions in enumerate(struc):
        postxt += '\n {} {:1.5f} {:1.5f} {:1.5f}'.format(positions[0], *positions[1])

    return postxt

In [23]:
def write_file(fname, text):
    with open(fname, 'w') as fin:
        fin.write(text)

### Make MD file.

In [24]:
# make MD text file for benzene
def get_md_text(pref, pseudo_dir, outdir, dt, nstep, pos):
    
    pos_txt = get_position_txt(pos)
    
    md_text = """&control
    calculation = 'md'
    prefix='{0}'    
    pseudo_dir = '{1}',
    outdir='{2}'
    dt = {3}
    nstep = {4}
 /
 &system
    ibrav = 6,
    celldm(1) = 32,
    celldm(3) = 0.83,
    nat = 12,
    ntyp = 2,
    ecutwfc = 25,
    ecutrho = 250,
    nosym = .true.
 /  
 &electrons
    diagonalization='david'
    mixing_mode = 'plain'
    mixing_beta = 0.7
    conv_thr =  1.0d-10
    !tqr=.true.
 /
 &ions
    pot_extrapolation = 'second-order'
    wfc_extrapolation = 'second-order'
 /
ATOMIC_SPECIES
C 12.01  C.pz-rrkjus.UPF
H 1.0    H.pz-rrkjus.UPF
{5}
K_POINTS {{gamma}}""".format(pref, pseudo_dir, outdir, dt, nstep, pos_txt)
    
    return md_text

In [25]:
# make text
md_text = get_md_text(pref, pseudo_dir, outdir, dt, nstep, struc_pert)
write_file(in_name, md_text)

### Make sh file.

In [26]:
def make_sh_text(ncpu, nodes, pw_loc, in_file, out_file, npool, partition, memory, email):

    sh_text = """#!/bin/sh
#SBATCH -n {0}
#SBATCH -N {1}
#SBATCH -t 1-00:00
#SBATCH -e test.err
#SBATCH -p {6}
#SBATCH -o test.out
#SBATCH --mem-per-cpu={7}
#SBATCH --mail-type=ALL
#SBATCH --mail-user={8}

module load gcc/4.9.3-fasrc01 openmpi/2.1.0-fasrc01
module load python/3.6.3-fasrc01

MPI="mpirun"
PW="{2}"

$MPI $PW -npool {5} < {3} > {4}
""".format(ncpu, nodes, pw_loc, in_file, out_file, npool, partition, memory, email)
    
    return sh_text

In [27]:
# make sh text
sh_text = make_sh_text(ncpu, nodes, pw_loc, in_name, out_name, npool, partition, memory, email)
write_file(sh_name, sh_text)